In [1]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

%matplotlib inline

In [2]:
from datasets import load_dataset

# data_en = load_dataset('head_qa', 'en')
data_es = load_dataset('head_qa', 'es' )
data_es

ModuleNotFoundError: No module named 'datasets'

In [ ]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [ ]:
for d in training:
    print(d)
    break;

In [ ]:
def parse_training(training)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, x_size, n_classes): 
        super(LogisticRegression, self).__init__()             
        self.linear = nn.Linear(x_size, n_classes)
        
    def forward(self, x):
        return self.linear(x)